<a href="https://colab.research.google.com/github/aruaru0/Foursquare-Location-Matching/blob/main/FourSqure_CV%E7%A2%BA%E8%AA%8D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# for colab

In [1]:
import sys
is_colab = False
if "google.colab" in sys.modules:
  is_colab = True

In [2]:
if is_colab:
  from google.colab import drive
  drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
if is_colab:
  !mkdir -p ~/.kaggle
  !cp /content/drive/MyDrive/datas/kaggle.json  ~/.kaggle/
  !chmod 600 /root/.kaggle/kaggle.json

In [4]:
if is_colab:
  !pip install kaggle
  !apt install unzip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
unzip is already the newest version (6.0-21ubuntu1.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [5]:
if is_colab:
  !mkdir input output

In [6]:
if is_colab:
  !kaggle competitions download -c foursquare-location-matching
  !unzip foursquare-location-matching.zip -d input/foursquare-location-matching

 99% 157M/159M [00:01<00:00, 138MB/s]
100% 159M/159M [00:01<00:00, 114MB/s]
Archive:  foursquare-location-matching.zip
  inflating: input/foursquare-location-matching/pairs.csv  
  inflating: input/foursquare-location-matching/sample_submission.csv  
  inflating: input/foursquare-location-matching/test.csv  
  inflating: input/foursquare-location-matching/train.csv  


In [7]:
if is_colab:
  !kaggle datasets download -d guoyonfan/binary-lgb-baseline
  !unzip /content/binary-lgb-baseline.zip -d input/binary-lgb-baseline

  0% 0.00/3.90M [00:00<?, ?B/s]
100% 3.90M/3.90M [00:00<00:00, 155MB/s]
Archive:  /content/binary-lgb-baseline.zip
  inflating: input/binary-lgb-baseline/improved_lgb_baseline.lgb  
  inflating: input/binary-lgb-baseline/lgb_baseline.lgb  


In [8]:
if is_colab:
  !cp input/binary-lgb-baseline/*  /content/drive/MyDrive/datas/foursquare

In [9]:
if is_colab:
  !pip install Levenshtein
  !pip install catboost
  !pip install cdifflib polyleven
else : 
  !cp -r ../input/foursquare-lightgbm-trained/polyleven-0.7/polyleven-0.7 .
  !cp -r ../input/foursquare-lightgbm-trained/cdifflib-1.2.5/cdifflib-1.2.5 .
  !pip install ./polyleven-0.7
  !pip install ./cdifflib-1.2.5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 258 kB 9.7 MB/s 
     |████████████████████████████████| 1.8 MB 51.0 MB/s 
     |████████████████████████████████| 101 kB 7.6 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 74 kB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for cdifflib: filename=cdifflib-1.2.5-cp37-cp37m-linux_x86_64.whl size=32978 sha256=8c3941fcbec19c802dd9114329129f48b07134da43ed0e955ca13ca3e1ebd16b
  Stored in directory: /root/.cache/pip/wheels/01/b2/78/56b07db93d847924bb3786d6b155675652cbe7f75103bb9c2f
  Created wheel for polyleven: filename=polyleven-0.7-cp37-cp37m-linux_x86_64.whl size=22688 sha256=55339db6b5e4c550e8c66a444b26cbe141b91e9a3526b5d2f88a3e007d22575d
  Stored in directory: /root/.cache/p

# change directory

In [10]:
if is_colab:
  %cd /content/output

/content/output


In [11]:
if is_colab:
    !pip install pykakasi
    !pip install fuzzywuzzy
else :
    !unzip ../input/foursquare-lightgbm-trained/pykakasi_deps.dontopenthiskaggle/pykakasi_deps.dontopenthiskaggle -d .
    !conda install ./pykakasi_deps/offline_pykakasi.tar.bz2
    !conda install ./pykakasi_deps/offline_jaconv.tar.bz2
    !conda install ./pykakasi_deps/offline_deprecated.tar.bz2
    !pip install ../input/foursquare-lightgbm-trained/fuzzywuzzy-0.18.0-py2.py3-none-any.whl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.4 MB 6.8 MB/s 
  Created wheel for jaconv: filename=jaconv-0.3-py3-none-any.whl size=15564 sha256=cc8e6a38be9d57bdd77626f44acd7bf83a374668f0ee885527893c72c6a7912c
  Stored in directory: /root/.cache/pip/wheels/8f/4f/c2/a2a3b14d0e94f855f4aa8887bf0267bee9ecfb8e62a9ee2d92
Successfully built jaconv
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## restart here to check

In [12]:
import sys
is_colab = False
if "google.colab" in sys.modules:
  is_colab = True

if is_colab:
  %cd /content/output

/content/output


In [13]:
## Imports
import warnings
warnings.filterwarnings('ignore')

import os
import gc
import time
import random
import Levenshtein
import difflib
import multiprocessing
import pandas as pd
import numpy as np
import lightgbm as lgb
from collections import Counter
from tqdm.auto import tqdm
from tqdm.notebook import tqdm

tqdm.pandas()

from sklearn.neighbors import KNeighborsRegressor
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

from catboost import CatBoostClassifier
import pykakasi

from fuzzywuzzy import fuzz
import math

from collections import defaultdict

from polyleven import levenshtein
from cdifflib import CSequenceMatcher

In [14]:
TRAIN_FEATURES  = [
      # 'kneighbors', 'kneighbors_country','kneighbors_categories', 
       'categories_index', 
       'latdiff',
       'londiff', 'manhattan', 'euclidean', 'haversine', 'name_sim',
       'name_gesh', 'name_leven', 'name_jaro', 'name_lcs', 'name_fuz',
       'name_len_diff', 'name_nleven', 'name_nlcsk', 'name_nlcs',
       'address_sim', 'address_gesh', 'address_leven', 'address_jaro',
       'address_lcs', 'address_fuz', 'address_len_diff', 'address_nleven',
       'address_nlcsk', 'address_nlcs', 'city_gesh', 'city_leven', 'city_jaro',
       'city_lcs', 'city_fuz', 'city_len_diff', 'city_nleven', 'city_nlcsk',
       'city_nlcs', 
       'phone_gesh', 'phone_jaro', 'phone_lcs',
       'categories_sim', 'categories_gesh', 'categories_leven',
       'categories_jaro', 'categories_lcs', 'categories_fuz',
       'categories_len_diff', 'categories_nleven', 'categories_nlcsk',
       'categories_nlcs',       
       'categories',
       'categories2_sim',
       'categories2_gesh', 'categories2_leven', 'categories2_jaro',
       'categories2_lcs', 'categories2_fuz', 'categories2_len_diff',
       'categories2_nleven', 'categories2_nlcsk', 'categories2_nlcs']


In [15]:
## Parameters
num_neighbors = 10
num_neighbors2 = 6
num_neighbors3 = 30
# num_neighbors = 10
# num_neighbors2 = 5
# num_neighbors3 = 22
SEED = 2022
THRESHOLD = 0.5
NUM_SPLIT = 10
# feat_columns = ['dist', 'name', 'address', 'city', 
#             'state', 'zip', 'url', 
#            'phone', 'categories', 'country', 'categories2']
# vec_columns = ['name', 'categories', 'address', 
#                'state', 'url', 'country', 'categories2']

feat_columns = ['dist', 'name', 'address', 'city', 
            'phone', 'categories', 'categories2']
vec_columns = ['name', 'categories', 'address', 'categories2']

#rec_columns = ['name', 'address', 'categories', 'address', 'phone']
rec_columns = ['name', 'address', 'categories2', 'address', 'phone']


def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    
seed_everything(SEED)

In [16]:
sel_cat = {
 'accessories stores': 0,
 'advertising agencies': 1,
 'african restaurants': 2,
 'airport lounges': 3,
 'airport services': 4,
 'airport terminals': 5,
 'airports': 6,
 'american restaurants': 7,
 'american restaurants, bars': 8,
 'american restaurants, burger joints': 9,
 'american restaurants, burger joints, fast food restaurants': 10,
 'antique shops': 11,
 'aquariums': 12,
 'arcades': 13,
 'argentinian restaurants': 14,
 'art galleries': 15,
 'art museums': 16,
 'arts & crafts stores': 17,
 'arts & crafts stores, convenience stores, discount stores': 18,
 'asian restaurants': 19,
 'asian restaurants, chinese restaurants': 20,
 'asian restaurants, indonesian restaurants': 21,
 'asian restaurants, malay restaurants': 22,
 'asian restaurants, thai restaurants': 23,
 'assisted living': 24,
 'athletics & sports': 25,
 'atms': 26,
 'atms, banks': 27,
 'atms, banks, financial or legal services': 28,
 'auditoriums': 29,
 'auto dealerships': 30,
 'auto dealerships, automotive shops': 31,
 'auto garages': 32,
 'auto garages, auto workshops, automotive shops': 33,
 'auto garages, automotive shops': 34,
 'auto workshops': 35,
 'auto workshops, automotive shops': 36,
 'automotive shops': 37,
 'automotive shops, gas stations': 38,
 'bagel shops': 39,
 'baggage claims': 40,
 'bakeries': 41,
 'bakeries, cafés': 42,
 'banks': 43,
 'banks, buildings': 44,
 'banks, financial or legal services': 45,
 'banks, offices': 46,
 'bars': 47,
 'bars, cafés': 48,
 'bars, nightclubs': 49,
 'bars, pubs': 50,
 'bars, restaurants': 51,
 'baseball fields': 52,
 'baseball stadiums': 53,
 'basketball courts': 54,
 'bbq joints': 55,
 'beach bars': 56,
 'beaches': 57,
 'beaches, surf spots': 58,
 'bed & breakfasts': 59,
 'bed & breakfasts, hotels': 60,
 'beer bars': 61,
 'beer gardens': 62,
 'big box stores': 63,
 'big box stores, department stores': 64,
 'big box stores, grocery stores, supermarkets': 65,
 'big box stores, supermarkets': 66,
 'bike rentals / bike shares': 67,
 'bike shops': 68,
 'bistros': 69,
 'boarding houses': 70,
 'boats or ferries': 71,
 'bookstores': 72,
 'border crossings': 73,
 'botanical gardens': 74,
 'boutiques': 75,
 'boutiques, clothing stores': 76,
 'boutiques, clothing stores, department stores': 77,
 "boutiques, clothing stores, women's stores": 78,
 "boutiques, women's stores": 79,
 'bowling alleys': 80,
 'brazilian restaurants': 81,
 'breakfast spots': 82,
 'breakfast spots, cafés': 83,
 'breweries': 84,
 'bridal shops': 85,
 'bridges': 86,
 'bubble tea shops': 87,
 'buddhist temples': 88,
 'buffets': 89,
 'buildings': 90,
 'buildings, offices': 91,
 'buildings, residential buildings (apartments / condos)': 92,
 'burger joints': 93,
 'burger joints, fast food restaurants': 94,
 'bus stations': 95,
 'bus stops': 96,
 'business centers': 97,
 'business services': 98,
 'butchers': 99,
 'cafeterias': 100,
 'cafés': 101,
 'cafés, coffee shops': 102,
 'cafés, dessert shops': 103,
 'cafés, restaurants': 104,
 'cafés, tea rooms': 105,
 'camera stores': 106,
 'campaign offices': 107,
 'campgrounds': 108,
 'candy stores': 109,
 'capitol buildings': 110,
 'car washes': 111,
 'casinos': 112,
 'castles': 113,
 'cemeteries': 114,
 'chinese restaurants': 115,
 'chinese restaurants, noodle houses': 116,
 'chiropractors': 117,
 'chocolate shops': 118,
 'churches': 119,
 'cigkofte places': 120,
 'city halls': 121,
 'clothing stores': 122,
 'clothing stores, department stores': 123,
 'clothing stores, sporting goods shops': 124,
 "clothing stores, women's stores": 125,
 'club houses': 126,
 'cocktail bars': 127,
 'coffee shops': 128,
 'coffee shops, donut shops': 129,
 'college academic buildings': 130,
 'college academic buildings, college classrooms': 131,
 'college academic buildings, universities': 132,
 'college administrative buildings': 133,
 'college arts buildings': 134,
 'college auditoriums': 135,
 'college bookstores': 136,
 'college cafeterias': 137,
 'college classrooms': 138,
 'college classrooms, universities': 139,
 'college communications buildings': 140,
 'college engineering buildings': 141,
 'college gyms': 142,
 'college labs': 143,
 'college libraries': 144,
 'college quads': 145,
 'college rec centers': 146,
 'college residence halls': 147,
 'college science buildings': 148,
 'college technology buildings': 149,
 'college theaters': 150,
 'colleges & universities': 151,
 'comedy clubs': 152,
 'comfort food restaurants': 153,
 'community centers': 154,
 'community colleges': 155,
 'concert halls': 156,
 'conference rooms': 157,
 'conferences': 158,
 'construction & landscaping': 159,
 'convenience stores': 160,
 'convenience stores, gas stations': 161,
 'convenience stores, grocery stores': 162,
 'convenience stores, snack places': 163,
 'convention centers': 164,
 'corporate cafeterias': 165,
 'cosmetics shops': 166,
 'cosmetics shops, pharmacies': 167,
 'cosmetics shops, salons / barbershops': 168,
 'courthouses': 169,
 'coworking spaces': 170,
 'coworking spaces, offices': 171,
 'credit unions': 172,
 'cultural centers': 173,
 'cupcake shops': 174,
 'dance studios': 175,
 'daycares': 176,
 'delis / bodegas': 177,
 "dentist's offices": 178,
 'department stores': 179,
 'department stores, shopping malls': 180,
 'design studios': 181,
 'dessert shops': 182,
 'dessert shops, ice cream shops': 183,
 'dim sum restaurants': 184,
 'diners': 185,
 'discount stores': 186,
 'discount stores, supermarkets': 187,
 'distribution centers': 188,
 'dive bars': 189,
 "doctor's offices": 190,
 "doctor's offices, hospitals": 191,
 "doctor's offices, medical centers": 192,
 'dog runs': 193,
 'doner restaurants': 194,
 'donut shops': 195,
 'driving schools': 196,
 'drugstores': 197,
 'drugstores, pharmacies': 198,
 'dumpling restaurants': 199,
 'eastern european restaurants': 200,
 'electronics stores': 201,
 'electronics stores, mobile phone shops': 202,
 'elementary schools': 203,
 'embassies / consulates': 204,
 'emergency rooms': 205,
 'emergency rooms, hospitals': 206,
 'ethiopian restaurants': 207,
 'event spaces': 208,
 'eye doctors': 209,
 'eye doctors, optical shops': 210,
 'factories': 211,
 'factories, offices': 212,
 'fairs': 213,
 'farmers markets': 214,
 'farms': 215,
 'fast food restaurants': 216,
 'fast food restaurants, fried chicken joints': 217,
 'fast food restaurants, fried chicken joints, restaurants': 218,
 'fast food restaurants, restaurants, sandwich places': 219,
 'fast food restaurants, sandwich places': 220,
 'festivals': 221,
 'fields': 222,
 'filipino restaurants': 223,
 'financial or legal services': 224,
 'financial or legal services, insurance offices': 225,
 'fire stations': 226,
 'fish & chips shops': 227,
 'flea markets': 228,
 'flower shops': 229,
 'food': 230,
 'food & drink shops': 231,
 'food courts': 232,
 'food trucks': 233,
 'football stadiums': 234,
 'forests': 235,
 'fraternity houses': 236,
 'french restaurants': 237,
 'fried chicken joints': 238,
 'frozen yogurt shops': 239,
 'fruit & vegetable stores': 240,
 'funeral homes': 241,
 'furniture / home stores': 242,
 'gaming cafes': 243,
 'garden centers': 244,
 'garden centers, hardware stores': 245,
 'gardens': 246,
 'gas stations': 247,
 'gastropubs': 248,
 'gay bars': 249,
 'general colleges & universities': 250,
 'general entertainment': 251,
 'general travel': 252,
 'german restaurants': 253,
 'gift shops': 254,
 'golf courses': 255,
 'gourmet shops': 256,
 'government buildings': 257,
 'government buildings, offices': 258,
 'greek restaurants': 259,
 'grocery stores': 260,
 'grocery stores, supermarkets': 261,
 'gyms': 262,
 'gyms or fitness centers': 263,
 'gyms, gyms or fitness centers': 264,
 'harbors / marinas': 265,
 'hardware stores': 266,
 'health & beauty services': 267,
 'health & beauty services, salons / barbershops': 268,
 'health food stores': 269,
 'high schools': 270,
 'historic sites': 271,
 'history museums': 272,
 'hobby shops': 273,
 'home services & repairs': 274,
 'hookah bars': 275,
 'hospital wards': 276,
 'hospitals': 277,
 'hospitals, medical centers': 278,
 'hostels': 279,
 'hot dog joints': 280,
 'hot springs': 281,
 'hotel bars': 282,
 'hotels': 283,
 'hotels, motels': 284,
 'hotels, resorts': 285,
 'hotpot restaurants': 286,
 'housing developments': 287,
 'housing developments, residential buildings (apartments / condos)': 288,
 'ice cream shops': 289,
 'indian restaurants': 290,
 'indie movie theaters': 291,
 'indonesian restaurants': 292,
 'industrial estates': 293,
 'insurance offices': 294,
 'internet cafes': 295,
 'it services': 296,
 'italian restaurants': 297,
 'italian restaurants, pizza places': 298,
 'japanese curry restaurants': 299,
 'japanese restaurants': 300,
 'japanese restaurants, sake bars': 301,
 'japanese restaurants, sushi restaurants': 302,
 'jewelry stores': 303,
 'juice bars': 304,
 'karaoke bars': 305,
 'kebab restaurants': 306,
 'kids stores': 307,
 'korean restaurants': 308,
 'lakes': 309,
 'language schools': 310,
 'laundry services': 311,
 'law schools': 312,
 'lawyers': 313,
 'libraries': 314,
 'light rail stations': 315,
 'lighthouses': 316,
 'lingerie stores': 317,
 "lingerie stores, women's stores": 318,
 'liquor stores': 319,
 'locksmiths': 320,
 'lounges': 321,
 'malay restaurants': 322,
 'markets': 323,
 'martial arts schools': 324,
 'massage studios': 325,
 'massage studios, spas': 326,
 'mattress stores': 327,
 'medical centers': 328,
 'medical labs': 329,
 'medical schools': 330,
 'mediterranean restaurants': 331,
 'meeting rooms': 332,
 "men's stores": 333,
 'metro stations': 334,
 'mexican restaurants': 335,
 'middle eastern restaurants': 336,
 'middle schools': 337,
 'military bases': 338,
 'miscellaneous shops': 339,
 'mobile phone shops': 340,
 'monuments / landmarks': 341,
 'mosques': 342,
 'motels': 343,
 'motorcycle shops': 344,
 'mountains': 345,
 'movie theaters': 346,
 'movie theaters, multiplexes': 347,
 'multiplexes': 348,
 'museums': 349,
 'music festivals': 350,
 'music stores': 351,
 'music venues': 352,
 'nail salons': 353,
 'nail salons, salons / barbershops': 354,
 'national parks': 355,
 'newsstands': 356,
 'nightclubs': 357,
 'non-profits': 358,
 'noodle houses': 359,
 'noodle houses, thai restaurants': 360,
 'nursery schools': 361,
 'offices': 362,
 'offices, tech startups': 363,
 'optical shops': 364,
 'other events': 365,
 'other great outdoors': 366,
 'other nightlife': 367,
 'other repair shops': 368,
 'paper / office supplies stores': 369,
 'parking': 370,
 'parks': 371,
 'parks, playgrounds': 372,
 'pastry shops': 373,
 'pedestrian plazas': 374,
 'performing arts venues': 375,
 'pet services': 376,
 'pet stores': 377,
 'pharmacies': 378,
 'photography labs': 379,
 'photography labs, photography studios': 380,
 'photography studios': 381,
 'physical therapists': 382,
 'piers': 383,
 'pizza places': 384,
 'pizza places, restaurants': 385,
 'playgrounds': 386,
 'plazas': 387,
 'police stations': 388,
 'pool halls': 389,
 'pools': 390,
 'portuguese restaurants': 391,
 'post offices': 392,
 'print shops': 393,
 'private schools': 394,
 'professional & other places': 395,
 'public art': 396,
 'pubs': 397,
 'racetracks': 398,
 'radio stations': 399,
 'ramen restaurants': 400,
 'real estate offices': 401,
 'record shops': 402,
 'recreation centers': 403,
 'recycling facilities': 404,
 'rental car locations': 405,
 'residential buildings (apartments / condos)': 406,
 'resorts': 407,
 'rest areas': 408,
 'restaurants': 409,
 'rivers': 410,
 'rock clubs': 411,
 'sake bars': 412,
 'salad places': 413,
 'salons / barbershops': 414,
 'sandwich places': 415,
 'scenic lookouts': 416,
 'schools': 417,
 'seafood restaurants': 418,
 'shipping stores': 419,
 'shoe stores': 420,
 'shoe stores, sporting goods shops': 421,
 'shopping malls': 422,
 'shopping plazas': 423,
 'shops & services': 424,
 'shrines': 425,
 'ski areas': 426,
 'smoke shops': 427,
 'snack places': 428,
 'soba restaurants': 429,
 'soccer fields': 430,
 'soccer stadiums': 431,
 'social clubs': 432,
 'som tum restaurants': 433,
 'sorority houses': 434,
 'soup places': 435,
 'spanish restaurants': 436,
 'spas': 437,
 'speakeasies': 438,
 'spiritual centers': 439,
 'sporting goods shops': 440,
 'sports bars': 441,
 'sports clubs': 442,
 'stables': 443,
 'stadiums': 444,
 'steakhouses': 445,
 'storage facilities': 446,
 'strip clubs': 447,
 'student centers': 448,
 'supermarkets': 449,
 'surf spots': 450,
 'sushi restaurants': 451,
 'taco places': 452,
 'tailor shops': 453,
 'tanning salons': 454,
 'tapas restaurants': 455,
 'tattoo parlors': 456,
 'tea rooms': 457,
 'tech startups': 458,
 'temples': 459,
 'tennis courts': 460,
 'thai restaurants': 461,
 'theaters': 462,
 'theme park rides/attractions': 463,
 'theme parks': 464,
 'thrift / vintage stores': 465,
 'tourist information centers': 466,
 'toy / game stores': 467,
 'trade schools': 468,
 'trails': 469,
 'train stations': 470,
 'tram stations': 471,
 'transportation services': 472,
 'travel agencies': 473,
 'travel lounges': 474,
 'tunnels': 475,
 'turkish restaurants': 476,
 'tv stations': 477,
 'udon restaurants': 478,
 'universities': 479,
 'urgent care centers': 480,
 'vacation rentals': 481,
 'vegetarian / vegan restaurants': 482,
 'veterinarians': 483,
 'video game stores': 484,
 'video stores': 485,
 'vietnamese restaurants': 486,
 'warehouse stores': 487,
 'warehouses': 488,
 'water parks': 489,
 'waterfalls': 490,
 'waterfronts': 491,
 'wedding halls': 492,
 'wine bars': 493,
 'wine shops': 494,
 'wineries': 495,
 'wings joints': 496,
 "women's stores": 497,
 'yoga studios': 498,
 'zoos': 499}

In [17]:
sel_cat = {y.lower() : sel_cat[y] for y in sel_cat}

In [18]:
%load_ext Cython

In [19]:
# %%cython
# def LCS(str S, str T):
#     cdef int i, j
#     cdef list dp = [[0] * (len(T) + 1) for _ in range(len(S) + 1)]
#     for i in range(len(S)):
#         for j in range(len(T)):
#             dp[i + 1][j + 1] = max(dp[i][j] + (S[i] == T[j]), dp[i + 1][j], dp[i][j + 1], dp[i + 1][j + 1])
#     return dp[len(S)][len(T)]

In [20]:
%%cython
import numpy as np  # noqa
cpdef int LCS(str S, str T):
    cdef int i, j
    cdef int cost
    cdef int v1,v2,v3,v4
    cdef int[:, :] dp = np.zeros((len(S) + 1, len(T) + 1), dtype=np.int32)
    for i in range(len(S)):
        for j in range(len(T)):
            cost = (int)(S[i] == T[j])
            v1 = dp[i, j] + cost
            v2 = dp[i + 1, j]
            v3 = dp[i, j + 1]
            v4 = dp[i + 1, j + 1]
            dp[i + 1, j + 1] = max((v1,v2,v3,v4))
    return dp[len(S)][len(T)]

In [21]:
def post_process(df):
    id2match = dict(zip(df['id'].values, df['matches'].str.split()))

    for base, match in df[['id', 'matches']].values:
        match = match.split()
        if len(match) == 1:        
            continue

        for m in match:
            if base not in id2match[m]:
                id2match[m].append(base)
    df['matches'] = df['id'].map(id2match).map(' '.join)
    return df 


# get manhattan distance
def manhattan(lat1, long1, lat2, long2):
    return np.abs(lat2 - lat1) + np.abs(long2 - long1)

# get haversine distance
def vectorized_haversine(lats1, lats2, longs1, longs2):
    radius = 6371
    dlat=np.radians(lats2 - lats1)
    dlon=np.radians(longs2 - longs1)
    a = np.sin(dlat/2) * np.sin(dlat/2) + np.cos(np.radians(lats1)) \
        * np.cos(np.radians(lats2)) * np.sin(dlon/2) * np.sin(dlon/2)
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    d = radius * c
    return d

In [22]:
def recall_simple(df):
    threshold = 2
    
    val2id_d = {}
    for col in rec_columns:
        temp_df = df[['id', col]]
        temp_df[col] = temp_df[col].str.lower()
        val2id = temp_df.groupby(col)['id'].apply(set).to_dict()
        val2id_d[col] = val2id
        del val2id
    
    cus_ids = []
    match_ids = []
    for vals in tqdm(df[rec_columns + ['id']].fillna('null').values):
        cus_id = vals[-1]
        match_id = []
        
        rec_match_count = []
        for i in range(len(rec_columns)):
            col = rec_columns[i]
            
            if vals[i] != 'null':
                rec_match_count += list(val2id_d[col][vals[i].lower()])
        rec_match_count = dict(Counter(rec_match_count))
        
        for k, v in rec_match_count.items():
            if v > threshold:
                match_id.append(k)
        
        cus_ids += [cus_id] * len(match_id)
        match_ids += match_id
    
    train_df = pd.DataFrame()
    train_df['id'] = cus_ids
    train_df['match_id'] = match_ids
    train_df = train_df.drop_duplicates()
    del cus_ids, match_ids
    
    num_data = len(train_df)
    num_data_per_id = num_data / train_df['id'].nunique()
    print('Num of data: %s' % num_data)
    print('Num of data per id: %s' % num_data_per_id)
    
    return train_df

In [23]:
def recall_knn(df, Neighbors = 10, Neighbors2 = 10, Neighbors3 = 10 ):
    print('Start knn grouped by country')
    train_df_country = []
    for country, country_df in tqdm(df.groupby('country')):
        country_df = country_df.reset_index(drop = True)

        neighbors = min(len(country_df), Neighbors)
        knn = KNeighborsRegressor(n_neighbors = neighbors,
                                    metric = 'haversine',
                                    n_jobs = -1)
        knn.fit(country_df[['latitude','longitude']].apply(lambda dr : (dr*math.pi)/180),
                country_df.index)
        dists, nears = knn.kneighbors(
            country_df[['latitude', 'longitude']].apply(lambda dr : (dr*math.pi)/180), 
            return_distance = True)
        # knn.fit(country_df[['latitude','longitude']],
        #         country_df.index)
        # dists, nears = knn.kneighbors(
        #     country_df[['latitude', 'longitude']], 
        #     return_distance = True)

        for k in range(neighbors):            
            cur_df = country_df[['id']]
            cur_df['match_id'] = country_df['id'].values[nears[:, k]]
            # cur_df['kdist_country'] = dists[:, k]
            cur_df['kneighbors_country'] = k
            
            train_df_country.append(cur_df)
    train_df_country = pd.concat(train_df_country)
    
    #-------------------------------------------------
    print('Start knn grouped by categories')
    seld = sel_cat

    train_df_categories = []
  
    for categories, categories_df in tqdm(df.groupby('categories')):
        if categories not in seld : continue
        categories_df = categories_df.reset_index(drop = True)

        neighbors = min(len(categories_df), Neighbors2)
        knn = KNeighborsRegressor(n_neighbors = neighbors,
                                    metric = 'haversine',
                                    n_jobs = -1)
        knn.fit(categories_df[['latitude','longitude']].apply(lambda dr : (dr*math.pi)/180),
                categories_df.index)
        dists, nears = knn.kneighbors(
            categories_df[['latitude', 'longitude']].apply(lambda dr : (dr*math.pi)/180), 
            return_distance = True)
        # knn.fit(categories_df[['latitude','longitude']],
        #         categories_df.index)
        # dists, nears = knn.kneighbors(
        #     categories_df[['latitude', 'longitude']], 
        #     return_distance = True)

        for k in range(neighbors):            
            cur_df = categories_df[['id']]
            cur_df['match_id'] = categories_df['id'].values[nears[:, k]]
            # cur_df['kdist_categories'] = dists[:, k]
            cur_df['kneighbors_categories'] = k
            cur_df['categories_index'] = seld[categories]
            
            train_df_categories.append(cur_df)
    train_df_categories = pd.concat(train_df_categories)
    #-------------------------------------------------
    print('Start knn')
    train_df = []
    knn = NearestNeighbors(n_neighbors = Neighbors3)
    knn.fit(df[['latitude','longitude']].apply(lambda dr : (dr*math.pi)/180), df.index)
    dists, nears = knn.kneighbors(df[['latitude','longitude']].apply(lambda dr : (dr*math.pi)/180))
    # knn.fit(df[['latitude','longitude']], df.index)
    # dists, nears = knn.kneighbors(df[['latitude','longitude']])
    
    for k in range(Neighbors3):            
        cur_df = df[['id']]
        cur_df['match_id'] = df['id'].values[nears[:, k]]
        # cur_df['kdist'] = dists[:, k]
        cur_df['kneighbors'] = k
        train_df.append(cur_df)
    
    train_df = pd.concat(train_df)
    train_df = train_df.merge(train_df_country,
                                 on = ['id', 'match_id'],
                                 how = 'outer')
    #-------------------------------------------------
    train_df = train_df.merge(train_df_categories,
                                on = ['id', 'match_id'],
                                how = 'outer')

    del train_df_country

    # pos = train_df['kdist_country'].notna()
    # train_df.loc[pos, 'kdist'] = train_df[pos]['kdist_country']
    # pos = train_df['kdist_categories'].notna()
    # train_df.loc[pos, 'kdist'] = train_df[pos]['kdist_categories']
    # sel = ['id', 'match_id', 'kdist', 'kneighbors',
    #       'kneighbors_country', 'kneighbors_categories','categories_index']
    # train_df = train_df[sel]
    # train_df['kneighbors_categories'] = train_df['kneighbors_categories'].astype('category')

    return train_df

In [24]:
def add_features(df):    
    for col in tqdm(feat_columns):
        if col == 'dist':
            lat = data.loc[df['id']]['latitude'].values
            match_lat = data.loc[df['match_id']]['latitude'].values
            lon = data.loc[df['id']]['longitude'].values
            match_lon = data.loc[df['match_id']]['longitude'].values
            df['latdiff'] = (lat - match_lat)
            df['londiff'] = (lon - match_lon)
            df['manhattan'] = manhattan(lat, lon, match_lat, match_lon)
            df['euclidean'] = (df['latdiff'] ** 2 + df['londiff'] ** 2) ** 0.5
            df['haversine'] = vectorized_haversine(lat, match_lat, lon, match_lon)
            continue
        
        col_values = data.loc[df['id']][col].values.astype(str)
        matcol_values = data.loc[df['match_id']][col].values.astype(str)
        
        if col in vec_columns:
            tv_fit = tfidf_d[col]
            indexs = [id2index_d[i] for i in df['id']]
            match_indexs = [id2index_d[i] for i in df['match_id']]                    
            df[f'{col}_sim'] = tv_fit[indexs].multiply(tv_fit[match_indexs]).\
                                            sum(axis = 1).A.ravel()
        
        geshs = []
        levens = []
        jaros = []
        lcss = []
        fuzwuz = []
        for s, match_s in zip(col_values, matcol_values):
            if s != 'nan' and match_s != 'nan':                    
                geshs.append(CSequenceMatcher(None, s, match_s).ratio())
                levens.append(levenshtein(s, match_s))
                jaros.append(Levenshtein.jaro_winkler(s, match_s))
                lcss.append(LCS(str(s), str(match_s)))
                fuzwuz.append(fuzz.ratio(s, match_s))
            else:
                geshs.append(np.nan)
                levens.append(np.nan)
                jaros.append(np.nan)
                lcss.append(np.nan)
                fuzwuz.append(np.nan)
        
        df[f'{col}_gesh'] = geshs
        df[f'{col}_leven'] = levens
        df[f'{col}_jaro'] = jaros
        df[f'{col}_lcs'] = lcss
        df[f'{col}_fuz'] = fuzwuz
        
        # add 2022.06.26
        # print(col)
        if col == 'categories' : 
          cats = []
          for s, match_s in zip(col_values, matcol_values):
            cats.append(categorical_similarity(s, match_s))
            # print(s,":", match_s, categorical_similarity(s, match_s))
          df[f'{col}'] = cats


        if col not in ['phone', 'zip']:
            df[f'{col}_len'] = list(map(len, col_values))
            df[f'match_{col}_len'] = list(map(len, matcol_values)) 
            df[f'{col}_len_diff'] = np.abs(df[f'{col}_len'] - df[f'match_{col}_len']) /\
                                        df[f'{col}_len'] 
            df[f'{col}_nleven'] = df[f'{col}_leven'] / \
                                    df[[f'{col}_len', f'match_{col}_len']].max(axis = 1)
            
            df[f'{col}_nlcsk'] = df[f'{col}_lcs'] / df[f'match_{col}_len']
            df[f'{col}_nlcs'] = df[f'{col}_lcs'] / df[f'{col}_len']
            
            df = df.drop(f'{col}_len', axis = 1)
            df = df.drop(f'match_{col}_len', axis = 1)
            gc.collect()
            
    return df

## alphabet convert

In [25]:
def convert_japanese_alphabet(df: pd.DataFrame):
    kakasi = pykakasi.kakasi()
    kakasi.setMode('H', 'a')  # Convert Hiragana into alphabet
    kakasi.setMode('K', 'a')  # Convert Katakana into alphabet
    kakasi.setMode('J', 'a')  # Convert Kanji into alphabet
    conversion = kakasi.getConverter()

    def convert(row):
        for column in ["name", "address", "city", "state"]:
            try:
                row[column] = conversion.do(row[column])
            except:
                pass
        return row

    df[df["country"] == "JP"] = df[df["country"] == "JP"].progress_apply(convert, axis=1)
    return df

In [26]:
def categorical_similarity(A, B):
    if not A or not B:
        return -1

    A = set(str(A).split(", "))
    B = set(str(B).split(", "))

    # Find intersection of two sets
    nominator = A.intersection(B)

    similarity_1 = len(nominator) / len(A)
    similarity_2 = len(nominator) / len(B)

    return max(similarity_1, similarity_2)

## for CV

In [27]:
# https://www.kaggle.com/code/columbia2131/foursquare-iou-metrics
def get_id2poi(input_df: pd.DataFrame) -> dict:
  return dict(zip(input_df['id'], input_df['point_of_interest']))

def get_poi2ids(input_df: pd.DataFrame) -> dict:
  return input_df.groupby('point_of_interest')['id'].apply(set).to_dict()

# KNNで選ばれたデータの正解のカバー率を計測する
def get_score(input_df: pd.DataFrame):
  scores = []
  for id_str, matches in zip(input_df['id'].to_numpy(), input_df['matches'].to_numpy()):
    targets = poi2ids[id2poi[id_str]]
    preds = set(matches.split())
    score = len((targets & preds)) / len((targets | preds))
    scores.append(score)
  scores = np.array(scores)
  return scores.mean()

In [28]:
cdict =  {
    'ATMs': 'Banks',
    'ATMs, Banks': 'Banks',
    'Airport Services': 'Airports',
    'Airport Terminals': 'Airports',
    'American Restaurants': 'Fast Food Restaurants',
    'Asian Restaurants': 'Chinese Restaurants',
    'Asian Restaurants, Chinese Restaurants': 'Chinese Restaurants',
    'Auto Dealerships': 'Automotive Shops',
    'Auto Dealerships, Automotive Shops': 'Auto Dealerships',
    'Auto Garages, Auto Workshops, Automotive Shops': 'Automotive Shops',
    'Auto Garages, Automotive Shops': 'Automotive Shops',
    'Auto Workshops, Automotive Shops': 'Automotive Shops',
    'Automotive Shops, Gas Stations': 'Automotive Shops',
    'Bakeries, Cafés': 'Bakeries',
    'Banks, Buildings': 'Banks',
    'Banks, Offices': 'Banks',
    'Beaches, Surf Spots': 'Beaches',
    'Bed & Breakfasts': 'Hotels',
    'Boutiques': 'Clothing Stores',
    'Buildings': 'Offices',
    'Buildings, Offices': 'Offices',
    'Buildings, Residential Buildings (Apartments / Condos)': 'Residential Buildings (Apartments / Condos)',
    'Burger Joints': 'Fast Food Restaurants',
    'Burger Joints, Fast Food Restaurants': 'Fast Food Restaurants',
    'Bus Stations': 'Bus Stops',
    'Business Services, Mobile Phone Shops': 'Mobile Phone Shops',
    'Cafés, Coffee Shops': 'Cafés',
    'Coffee Shops': 'Cafés',
    'College Academic Buildings': 'College Classrooms',
    'College Auditoriums': 'College Classrooms',
    'Convenience Stores, Gas Stations': 'Gas Stations',
    'Coworking Spaces': 'Offices',
    'Coworking Spaces, Offices': 'Offices',
    'Department Stores': 'Shopping Malls',
    'Discount Stores, Supermarkets': 'Grocery Stores',
    "Doctor's Offices": 'Hospitals',
    "Doctor's Offices, Medical Centers": "Doctor's Offices",
    'Drugstores': 'Pharmacies',
    'Drugstores, Pharmacies': 'Pharmacies',
    'Electronics Stores, Mobile Phone Shops': 'Mobile Phone Shops',
    'Fast Food Restaurants, Fried Chicken Joints': 'Fast Food Restaurants',
    'Fast Food Restaurants, Restaurants, Sandwich Places': 'Sandwich Places',
    'Fast Food Restaurants, Sandwich Places': 'Sandwich Places',
    'Fried Chicken Joints': 'Fast Food Restaurants',
    'Furniture / Home Stores, Lighting Stores': 'Furniture / Home Stores',
    'Government Buildings': 'Offices',
    'Grocery Stores': 'Convenience Stores',
    'Grocery Stores, Supermarkets': 'Grocery Stores',
    'Gyms or Fitness Centers': 'Gyms',
    'Gyms, Gyms or Fitness Centers': 'Gyms',
    'Health & Beauty Services, Salons / Barbershops': 'Salons / Barbershops',
    'High Schools': 'College Classrooms',
    'Hospital Wards': 'Hospitals',
    'Hospitals, Medical Centers': 'Hospitals',
    'Hotels': 'Residential Buildings (Apartments / Condos)',
    'Hotels, Resorts': 'Hotels',
    'Housing Developments': 'Residential Buildings (Apartments / Condos)',
    'Housing Developments, Residential Buildings (Apartments / Condos)': 'Residential Buildings (Apartments / Condos)',
    'Indonesian Restaurants': 'Chinese Restaurants',
    'Italian Restaurants': 'Cafés',
    'Italian Restaurants, Pizza Places': 'Italian Restaurants',
    'Japanese Restaurants, Sake Bars': 'Japanese Restaurants',
    'Japanese Restaurants, Sushi Restaurants': 'Japanese Restaurants',
    'Light Rail Stations': 'Metro Stations',
    'Massage Studios, Spas': 'Spas',
    'Medical Centers': "Doctor's Offices",
    'Middle Schools': 'High Schools',
    'Motels': 'Residential Buildings (Apartments / Condos)',
    'Movie Theaters, Multiplexes': 'Movie Theaters',
    'Multiplexes': 'Movie Theaters',
    'Nail Salons, Salons / Barbershops': 'Salons / Barbershops',
    'Noodle Houses': 'Chinese Restaurants',
    'Offices': 'Banks',
    'Other Great Outdoors': 'Parks',
    'Pizza Places, Restaurants': 'Pizza Places',
    'Pubs': 'Bars',
    'Ramen Restaurants': 'Noodle Houses',
    'Resorts': 'Hotels',
    'Restaurants': 'Cafés',
    'Schools': 'High Schools',
    'Shopping Malls': 'Offices',
    'Soccer Stadiums': 'Soccer Fields',
    'Supermarkets': 'Grocery Stores',
    'Sushi Restaurants': 'Japanese Restaurants',
    'Tech Startups': 'Offices',
    'Thai Restaurants': 'Chinese Restaurants',
    'Train Stations': 'Metro Stations',
    'Universities': 'College Classrooms',
    "Women's Stores": 'Clothing Stores'}


In [29]:
cdict = {y.lower() : cdict[y].lower() for y in cdict}

In [30]:
def f(x) :
  if type(x)==float and math.isnan(x) : return x
  if x in cdict : return cdict[x]
  return x

## data load

In [31]:
def get_train(select, pair) :
  data = pd.read_csv('../input/foursquare-location-matching/train.csv')
  tv_ids_d = np.load('/content/drive/MyDrive/datas/foursquare/traindata0626/tv_ids_d.npy', allow_pickle=True).item()
  data = data.set_index('id')
  data = data.loc[tv_ids_d['valid_ids']]
  
  num_sel = int(len(data)*select)
  num_pair = int(num_sel*pair)

  pol_cnt = defaultdict(int)
  for e in data['point_of_interest'] :
    pol_cnt[e] += 1

  selected_poi = set()
  cnt = 0
  for e in pol_cnt :
    if pol_cnt[e] <= 1 : continue
    selected_poi.add(e)
    cnt += pol_cnt[e]
    if cnt >= num_pair : break

  for e in pol_cnt:
    if pol_cnt[e] > 1 : continue
    selected_poi.add(e)
    cnt += pol_cnt[e]
    if cnt >= num_sel : break

  len(selected_poi), cnt, num_sel, num_pair

  selected = []
  for e in data['point_of_interest'] :
    if e in selected_poi :
      selected.append(True)
    else: 
      selected.append(False)

  data = data[selected]
  print("selected = ", len(data))
  return data


In [32]:
from sklearn.model_selection import GroupKFold

## Dada process
if is_colab:
    data = pd.read_csv('../input/foursquare-location-matching/train.csv')
    tv_ids_d = np.load('/content/drive/MyDrive/datas/foursquare/traindata0626/tv_ids_d.npy', allow_pickle=True).item()
    data = data.set_index('id')
    data = data.loc[tv_ids_d['valid_ids']]
    # data = data.iloc[idx] 

    # data = get_train(1/20, 0.35) 
    data = data.reset_index()
    # id --> poi変換用の辞書. １対１の関係
    id2poi = get_id2poi(data)
    # poi --> id変換用の辞書辞書. １対多の関係
    poi2ids = get_poi2ids(data)   
    data = data.drop('point_of_interest', axis = 1)

else :
    data = pd.read_csv('../input/foursquare-location-matching/test.csv')

    if len(data) < 20:
        data = pd.read_csv('../input/foursquare-location-matching/train.csv',
                          nrows = 100)
        data = data.drop('point_of_interest', axis = 1)
   
#need to new dataset
# data = convert_japanese_alphabet(data)


# sort categories 0626
print("sort categories add 06/26")
cat = []
for e in data['categories'] :
  if type(e) == float :
    cat.append(e)
    continue
  e = e.split(", ")
  e.sort()
  cat.append(", ".join(e))
data['categories'] = cat
del cat
#-------------------
print("add categories2")
data['categories2'] = data['categories'].apply(f)

# special process
def get_lower(x):
    try:
        return x.lower()
    except:
        return x

for col in data.columns:
    if data[col].dtype == object and col != 'id':
        data[col] = data[col].apply(get_lower)
        
id2index_d = dict(zip(data['id'].values, data.index))

tfidf_d = {}
for col in vec_columns:
    tfidf = TfidfVectorizer()
    tv_fit = tfidf.fit_transform(data[col].fillna('nan'))
    tfidf_d[col] = tv_fit

out_df = pd.DataFrame()
out_df['id'] = data['id'].unique().tolist()
out_df['match_id'] = out_df['id']

test_data_simple = recall_simple(data)
test_data = recall_knn(data, num_neighbors, num_neighbors2, num_neighbors3)

print('train data by knn: %s' % len(test_data))
test_data = test_data.merge(test_data_simple,
                             on = ['id', 'match_id'],
                             how = 'outer')
del test_data_simple
gc.collect()

data = data.set_index('id')
print('Num of unique id: %s' % test_data['id'].nunique())
print('Num of test data: %s' % len(test_data))
print(test_data.sample(5))

sort categories add 06/26
add categories2


  0%|          | 0/379604 [00:00<?, ?it/s]

Num of data: 797283
Num of data per id: 3.205660384944775
Start knn grouped by country


  0%|          | 0/199 [00:00<?, ?it/s]

Start knn grouped by categories


  0%|          | 0/18747 [00:00<?, ?it/s]

Start knn
train data by knn: 12568899
Num of unique id: 379604
Num of test data: 13064153
                        id          match_id  kneighbors  kneighbors_country  \
11100541  E_3deaa43b560235  E_e0571aac033e46        29.0                 NaN   
4470334   E_c6c81f5a5b086e  E_87890a2170054d        11.0                 NaN   
8587665   E_9f1a0cfe40d9d4  E_4ac4772e66e463        22.0                 NaN   
1385144   E_a5dea920478009  E_3a67fa75e00dda         3.0                 3.0   
7835791   E_a41143b0f5510a  E_677acc4d39bc18        20.0                 NaN   

          kneighbors_categories  categories_index  
11100541                    NaN               NaN  
4470334                     NaN               NaN  
8587665                     NaN               NaN  
1385144                     NaN               NaN  
7835791                     NaN               NaN  


In [33]:
"""
22
train data by knn: 480978
Num of unique id: 18980
Num of test data: 482329

30
Start knn
train data by knn: 630620
Num of unique id: 18980
Num of test data: 631969 -> 421648

30 
Start knn
train data by knn: 12568899
Num of unique id: 379604
Num of test data: 13064153
"""

'\n22\ntrain data by knn: 480978\nNum of unique id: 18980\nNum of test data: 482329\n\n30\nStart knn\ntrain data by knn: 630620\nNum of unique id: 18980\nNum of test data: 631969 -> 421648\n\n30 \nStart knn\ntrain data by knn: 12568899\nNum of unique id: 379604\nNum of test data: 13064153\n'

## 重複パターンをid, match_id = x, y or y, x を見つけて除外（アンサンブル時はx＝yも消してよいかも

In [34]:
id2idx = {e:i for i, e in enumerate(data.index)}
id0 = [id2idx[e] for e in test_data['id']]
id1 = [id2idx[e] for e in test_data['match_id']]

sel = []
tot, cnt = 0,0
pair = set()
for i in tqdm(range(len(id0))) :
  x, y = id0[i], id1[i]
  # if y == x : 
  #   sel.append(False)
  if (y,x) in pair :
    sel.append(False)
    pair.remove((y, x))
  else :
    sel.append(True)
    pair.add((x, y))
    cnt+=1
  tot+=1

print("Total = {}　remain = {} data shrink rate = {}".format(tot, cnt, cnt/tot))
test_data = test_data[sel].reset_index()

del id2idx, id0, id1
gc.collect()

  0%|          | 0/13064153 [00:00<?, ?it/s]

Total = 13064153　remain = 8507941 data shrink rate = 0.651243214925606


0

## ---

In [35]:
data.sample(5)

,name,latitude,longitude,address,city,state,zip,country,url,phone,categories,categories2
id,,,,,,,,,,,,
E_89afee8bb40d46,u vejvodů,50.084212,14.419036,jilská 4,praha,prága,110 00,cz,http://www.restauraceuvejvodu.cz,776333877,"czech restaurants, pubs","czech restaurants, pubs"
E_40364f3c6af007,star market,42.407943,-71.234077,1070 lexington st,waltham,ma,02452,us,http://www.starmarket.com/pd/stores/ma/waltham...,+17818910615,"grocery stores, supermarkets","grocery stores, supermarkets"
E_eed81ea5566a36,ป้านิ ตลาดนัดโต้รุ่ง,12.571202,99.955566,NaN,หัวหิน,NaN,NaN,th,NaN,NaN,dessert shops,dessert shops
E_98547628c4a022,日本lv羊绒围巾价格-一比一原单a货奢侈品的,38.415602,-95.925751,sb中，高仿▊选款溦商呺 ↘ 1143 88б8↙▊原版超一仿高精仿法国的买家奢侈品包包围巾...,NaN,ks,NaN,us,NaN,NaN,ethiopian restaurants,ethiopian restaurants
E_c3916dca431604,metal shop,42.416163,27.698824,NaN,NaN,NaN,NaN,bg,NaN,NaN,hobby shops,hobby shops


In [36]:
# add_features(test_data.iloc[:10])

In [37]:
## Model load
mtype = [
    'lightgbm',
    'lightgbm',
    'lightgbm',
    'lightgbm',
    'lightgbm',
    # 'catboost',
    # 'catboost',
    # 'lightgbm',
]
models = [
    '/content/drive/MyDrive/datas/foursquare/lightgbm/lgbm0626_2_fold0.txt',
    '/content/drive/MyDrive/datas/foursquare/lightgbm/lgbm0626_2_fold1.txt',
    '/content/drive/MyDrive/datas/foursquare/lightgbm/lgbm0626_2_fold2.txt',
    '/content/drive/MyDrive/datas/foursquare/lightgbm/lgbm0626_2_fold3.txt',
    '/content/drive/MyDrive/datas/foursquare/lightgbm/lgbm0626_2_fold4.txt',
    # '/content/drive/MyDrive/datas/foursquare/lightgbm/lgbm0626_fold5.txt',
    # '/content/drive/MyDrive/datas/foursquare/lightgbm/lgbm_cat2_fold1.txt',
    # '/content/drive/MyDrive/datas/foursquare/lightgbm/lgbm_cat2_fold2.txt',
    # '/content/drive/MyDrive/datas/foursquare/lightgbm/lgbm_cat2_fold3.txt',
    # '/content/drive/MyDrive/datas/foursquare/lightgbm/lgbm_cat2_fold4.txt',
    # '/content/drive/MyDrive/datas/foursquare/lightgbm/lgbm_cat2_weighted2_fold0.txt',
    # '/content/drive/MyDrive/datas/foursquare/lightgbm/lgbm_cat2_weighted_fold1.txt',
    # '/content/drive/MyDrive/datas/foursquare/lightgbm/lgbm_cat2_fold2.txt',
    # '/content/drive/MyDrive/datas/foursquare/lightgbm/lgbm_cat2_fold3.txt',
    # '/content/drive/MyDrive/datas/foursquare/lightgbm/lgbm_cat2_fold4.txt',
    # '/content/drive/MyDrive/datas/foursquare/lightgbm/cat_cat2_fold0.cbm',
]
features = [
           TRAIN_FEATURES,
           TRAIN_FEATURES,
           TRAIN_FEATURES,
           TRAIN_FEATURES,
           TRAIN_FEATURES,
           ]

mix_model = []
for i, e in enumerate(mtype) :
    if e == 'catboost' :
      m = CatBoostClassifier()
      m.load_model(models[i])
      mix_model.append(m)
    else :
      mix_model.append(lgb.Booster(model_file = models[i]))

mix_model

In [38]:
# x = add_features(test_data.sample(5))

## prediction

In [39]:
%%time
## Prediction
count = 0
start_row = 0
pred_df = pd.DataFrame()
unique_id = test_data['id'].unique().tolist()
num_split_id = len(unique_id) // NUM_SPLIT
for k in range(1, NUM_SPLIT + 1):
    print('Current split: %s' % k)
    end_row = start_row + num_split_id
    if k < NUM_SPLIT:
        cur_id = unique_id[start_row : end_row]
        cur_data = test_data[test_data['id'].isin(cur_id)]
    else:
        cur_id = unique_id[start_row: ]
        cur_data = test_data[test_data['id'].isin(cur_id)]
    
    # add features & model prediction
    cur_data = add_features(cur_data)

    # print(cur_data.head())
    # cur_data['kdist_diff'] = (cur_data['kdist'] - cur_data['kdist_country']) /\
    #                             cur_data['kdist_country']
    # cur_data['kneighbors_mean'] = cur_data[['kneighbors', 'kneighbors_country']].mean(axis = 1)

    preds = []
    for num, model in enumerate(mix_model) :
      if mtype[num] == 'catboost' :
          res = model.predict_proba(cur_data[features[num]])[:,1]
      else :
          res = model.predict(cur_data[features[num]])#,predict_disable_shape_check=True)
      cur_data['pred{}'.format(num)] = res
      preds.append('pred{}'.format(num))

#    cur_pred_df = cur_data[cur_data['pred'] > THRESHOLD][['id', 'match_id']]
    cur_pred_df = cur_data[['id','match_id']+preds]
    pred_df = pd.concat([pred_df, cur_pred_df])
    
    start_row = end_row
    count += len(cur_data)

    del cur_data, cur_pred_df
    gc.collect()
print(count)

Current split: 1


  0%|          | 0/7 [00:00<?, ?it/s]

Current split: 2


  0%|          | 0/7 [00:00<?, ?it/s]

Current split: 3


...

Current split: 9


  0%|          | 0/7 [00:00<?, ?it/s]

Current split: 10


  0%|          | 0/7 [00:00<?, ?it/s]

8507941
CPU times: user 8h 16min 35s, sys: 2min 39s, total: 8h 19min 14s
Wall time: 4h 35min 7s


In [40]:
pred_df

,id,match_id,pred0,pred1,pred2,pred3,pred4
0,E_0000c566a81ea1,E_0000c566a81ea1,9.999998e-01,9.999996e-01,9.999997e-01,9.999990e-01,9.999999e-01
1,E_0000d9e584ed9f,E_0000d9e584ed9f,1.000000e+00,1.000000e+00,1.000000e+00,9.999997e-01,1.000000e+00
2,E_00011cca3f0bd6,E_00011cca3f0bd6,9.999998e-01,9.999997e-01,1.000000e+00,9.999976e-01,9.999999e-01
3,E_00015cd7e0227f,E_00015cd7e0227f,9.999945e-01,9.999969e-01,9.999971e-01,9.999962e-01,9.999991e-01
4,E_0001b37c96a0ff,E_0001b37c96a0ff,9.999995e-01,9.999982e-01,9.999991e-01,9.999993e-01,9.999996e-01
...,...,...,...,...,...,...,...
8507936,E_fe3b25a6be0358,E_fea0d55c95ba4c,3.137845e-08,2.500997e-08,4.173971e-08,4.059385e-07,8.489041e-08
8507937,E_fe6cafdec3ab4c,E_fe9a664b20f4ee,1.808393e-09,1.869678e-08,2.392633e-08,1.322557e-08,3.103922e-09
8507938,E_fea0d55c95ba4c,E_febee7ac1fecaa,5.235817e-08,1.115543e-07,1.922480e-07,5.259856e-07,2.281088e-07
8507939,E_fef4c4a94984f9,E_fef4c4a94984f9,9.999998e-01,9.999994e-01,9.999978e-01,9.999999e-01,9.999976e-01


In [41]:
n = len(mix_model)
if n == 5 :
  pred_df['pred']=(pred_df['pred0'] + pred_df['pred1'] + pred_df['pred2'] + pred_df['pred3'] + pred_df['pred4'])/5
elif n == 4 :
  pred_df['pred']=(pred_df['pred0'] + pred_df['pred1'] + pred_df['pred2'] + pred_df['pred3'])/4
elif n == 3 :
  pred_df['pred']=(pred_df['pred0']+pred_df['pred1']+pred_df['pred2'])/3
elif n == 2 :
  pred_df['pred']=(pred_df['pred0']+pred_df['pred1'])/2
else :
  pred_df['pred']=pred_df['pred0']

In [42]:
pred_df.sample(20)

,id,match_id,pred0,pred1,pred2,pred3,pred4,pred
4917788,E_97ddc39c2ca15f,E_c204d9153280b3,4.188305e-06,1.303564e-05,9.526750e-06,1.123626e-05,6.113790e-05,1.982497e-05
3093969,E_123d90c021ab18,E_a356f94cf551d2,8.445404e-07,6.268958e-06,9.220108e-07,7.034191e-06,7.387868e-06,4.491514e-06
1375242,E_fc59a15bd74a0a,E_ce35da630baa0f,1.621331e-04,2.555016e-05,5.255229e-06,1.703077e-05,6.982583e-05,5.595901e-05
5091070,E_4ef69ea2c79929,E_fd9fdd06138870,4.872607e-09,2.873703e-08,2.585774e-08,5.416160e-08,5.588773e-08,3.390334e-08
185741,E_7d16d2ce11ee40,E_7d16d2ce11ee40,9.999998e-01,9.999997e-01,9.999999e-01,9.999998e-01,9.999997e-01,9.999998e-01
3523854,E_d82e7db554a328,E_66a77c87d0582c,3.554584e-05,7.135948e-06,1.266539e-06,3.782256e-06,1.041080e-05,1.162828e-05
5199266,E_c1730dbc040b39,E_324a0808158e12,1.364028e-04,1.658354e-04,5.243177e-05,8.528732e-04,1.522537e-05,2.445537e-04
7767068,E_58d02c4d5ca675,E_508d944a786ba5,1.957961e-05,3.797471e-05,3.801453e-06,2.014384e-05,2.109485e-05,2.051889e-05
399678,E_0dd248295059cd,E_88a0a0e5811ba8,9.949403e-03,6.437030e-03,1.265089e-02,7.153017e-03,6.810723e-03,8.600212e-03
8317130,E_1abc637ba19761,E_36fa5eb405cb84,2.357647e-09,2.173015e-09,9.582969e-09,6.235759e-09,2.111515e-09,4.492181e-09


In [43]:
# if is_colab:
#   cnt = 0
#   tot = len(pred_df)
#   df_sel = pred_df[['id','match_id', 'pred']]
#   for i in range(len(pred_df)) :
#     x, y, p = df_sel.iloc[i]
#     px = id2poi[x]
#     py = id2poi[y]
#     if px == py and p > 0.5 : cnt += 1
#     if px != py and p < 0.5 : cnt += 1
#     if i!=0 and i%10000 == 0:
#       print(cnt, i, cnt/i)
#     # print(x, y, p)
#     # if i == 10:break
#   cnt, tot, cnt/tot

In [44]:
"""
551265 570000 0.9671315789473685 1FOLD 
551136 570000 0.9669052631578947 5FOLD
550725 570000 0.9661842105263158 3FLOD
"""

'\n551265 570000 0.9671315789473685 1FOLD \n551136 570000 0.9669052631578947 5FOLD\n550725 570000 0.9661842105263158 3FLOD\n'

In [45]:
pred_df = pred_df[pred_df['pred'] > THRESHOLD][['id', 'match_id']]
pred_df

,id,match_id
0,E_0000c566a81ea1,E_0000c566a81ea1
1,E_0000d9e584ed9f,E_0000d9e584ed9f
2,E_00011cca3f0bd6,E_00011cca3f0bd6
3,E_00015cd7e0227f,E_00015cd7e0227f
4,E_0001b37c96a0ff,E_0001b37c96a0ff
...,...,...
8507883,E_fb3d4b4d314104,E_fb3d4b4d314104
8507894,E_fbf60c7304a869,E_fbf60c7304a869
8507900,E_fc4a529b00f240,E_fc4a529b00f240
8507939,E_fef4c4a94984f9,E_fef4c4a94984f9


## POST PROCESS

In [46]:
## Submission    
out_df = pd.concat([out_df, pred_df])
out_df = out_df.groupby('id')['match_id'].\
                        apply(list).reset_index()
out_df['matches'] = out_df['match_id'].apply(lambda x: ' '.join(set(x)))
out_df = post_process(out_df)
print('Unique id: %s' % len(out_df))
print(out_df.head())

out_df = out_df[['id', 'matches']]

Unique id: 379604
                 id                                           match_id  \
0  E_0000c566a81ea1  [E_0000c566a81ea1, E_0000c566a81ea1, E_8d58f31...   
1  E_0000d9e584ed9f               [E_0000d9e584ed9f, E_0000d9e584ed9f]   
2  E_00011cca3f0bd6  [E_00011cca3f0bd6, E_00011cca3f0bd6, E_aeffddf...   
3  E_00015cd7e0227f               [E_00015cd7e0227f, E_00015cd7e0227f]   
4  E_0001b37c96a0ff               [E_0001b37c96a0ff, E_0001b37c96a0ff]   

                             matches  
0  E_0000c566a81ea1 E_8d58f3151bae83  
1  E_0000d9e584ed9f E_caad79f6ed7c44  
2  E_00011cca3f0bd6 E_aeffddf31a4b9e  
3                   E_00015cd7e0227f  
4                   E_0001b37c96a0ff  


In [47]:
out_df = post_process(out_df)
if is_colab:
  print(f"CV: {get_score(out_df):.6f}")

CV: 0.854618


In [48]:
out_df = post_process(out_df)
if is_colab:
  print(f"CV: {get_score(out_df):.6f}")

CV: 0.854618


In [49]:
out_df[['id', 'matches']].to_csv('submission.csv', index = False)

## show CV

In [50]:
if is_colab:
  print(f"CV: {get_score(out_df):.6f}")

CV: 0.854618


In [51]:
""" NEW 06/26
35% CV: 0.875048   ※Trainデータの割合
42% CV: 0.850410
50% CV: 0.816657
35% CV: 0.880970 new 3fold model 6/27 version　Wall time: 7min 16s
35% CV: 0.881284 new 3fold model 6/28 version　Wall time: Wall time: 17min 7s

CV: 0.875414
CV: 0.875414

0.871856 22 , 0.872733 25 , 0.873689 30 , 0.874907 30-3fold 0.874981 30-5fold
"""

' NEW 06/26\n35% CV: 0.875048   ※Trainデータの割合\n42% CV: 0.850410\n50% CV: 0.816657\n35% CV: 0.880970 new 3fold model 6/27 version\u3000Wall time: 7min 16s\n35% CV: 0.881284 new 3fold model 6/28 version\u3000Wall time: Wall time: 17min 7s\n\nCV: 0.875414\nCV: 0.875414\n\n0.871856 22 , 0.872733 25 , 0.873689 30 , 0.874907 30-3fold 0.874981 30-5fold\n'

In [52]:
# LB:    0.858 0.864
# 20% -> 0.93x
# 40% -> 0.874
# 45% -> 0.858 0.8583
# 50% -> 0.840
"""
CV: 0.856132
CV: 0.856653 --- 5fold
CV: 0.858016 --- catboost
CV: 0.855997 --- catboost 40000
CV: 0.855418 --- new lightgbm 
CV: 0.857235 --- new lightgbm 5fold
CV: 0.850607 --- new lightgbm 5fold all data
"""

'\nCV: 0.856132\nCV: 0.856653 --- 5fold\nCV: 0.858016 --- catboost\nCV: 0.855997 --- catboost 40000\nCV: 0.855418 --- new lightgbm \nCV: 0.857235 --- new lightgbm 5fold\nCV: 0.850607 --- new lightgbm 5fold all data\n'

In [53]:
# CV: 0.914089
# CV: 0.913176 5fold
# CV: 0.908401 dart model

In [54]:
# CV: 0.914605 no weight no kanji(miss)
#------------------------------------------------------------------
# CV: 0.934487 weighted sqrt(neg/pos) no kanji(miss)
# CV: 0.934492 weighted srqt(neg/pos) 2fold no kanji(miss)
# CV: 0.931351 wdithged sqrt(neg/pos) light version no kanji(miss)
# CV: 0.934244 weighted sqrt(neg/pos) 
# CV: 0.934954 weighted sqrt(neg/pos) middle version kanji 

In [55]:
# CV: 0.907733　LB:0.856

In [56]:
# CV: 0.907162 1fold
# CV: 0.911148 1fold weighted
# CV : 0.909899 2fold (non weight + weighted)
# CV: 0.909890 1fold weight = 2
